In [11]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

import torch

In [4]:
# ToDo find a model that runs faster
model_name = "HuggingFaceH4/zephyr-7b-beta"

# If you have a GPU available you can use this code
# bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
# model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

# Instantiate one of the model classes of the library (with a causal language modeling head) from a pretrained model.
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True)
# Instantiate one of the tokenizer classes of the library from a pretrained model vocabulary.
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 8/8 [01:21<00:00, 10.15s/it]


In [8]:
# First, create a text_generation pipeline using the loaded model and its tokenizer.
# The pipeline() makes it simple to use any model from the Hub for inference on language, and other tasks.

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [9]:
# Create a prompt template
# this should follow the format of the model, so make sure to use the appropriate formatting.
prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

# You can also use tokenizer.apply_chat_template to convert a list of messages (as dicts: {'role': 'user', 'content': '(...)'})
# into a string with the appropriate chat format.

In [13]:
# Finally, we need to combine the llm_chain with the retriever to create a RAG chain.
# We pass the original question through to the final generation step, as well as the retrieved context docs
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

db = FAISS.load_local(folder_path="faiss_db/", embeddings=embeddings, index_name="nasa_index", allow_dangerous_deserialization=True)

# This retriever returns the top 4 similar chunks
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

In [16]:
question = "Tell me about auroras."

# Answer with just the model itself, no context added. Took too long to compute.
#llm_chain.invoke({"context": "", "question": question})

In [17]:
# A RAG response considering our NASA context
rag_chain.invoke(question)

KeyboardInterrupt: 